In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# --- PASO 1: Cargar los Datos ---
# Asegúrate de que los archivos CSV estén en el mismo directorio que tu script/notebook
# o proporciona la ruta completa.

try:
    df_data = pd.read_csv('API_SM.POP.NETM_DS2_en_csv_v2_13382.csv', skiprows=4) # Saltar las primeras filas descriptivas
    df_meta_country = pd.read_csv('Metadata_Country_API_SM.POP.NETM_DS2_en_csv_v2_13382.csv')
    df_meta_indicator = pd.read_csv('Metadata_Indicator_API_SM.POP.NETM_DS2_en_csv_v2_13382.csv')

    print("Archivos cargados exitosamente.")
except FileNotFoundError as e:
    print(f"Error: No se encontró el archivo {e.filename}. Asegúrate de que esté en la ruta correcta.")
    exit() # Salir si no se pueden cargar los datos

# --- PASO 2: Entender el Indicador ---
# Ver la descripción del indicador que estamos usando (SM.POP.NETM)
print("\n--- Información del Indicador ---")
print(df_meta_indicator[df_meta_indicator['INDICATOR_CODE'] == 'SM.POP.NETM'])

# --- PASO 3: Extraer y Preparar los Datos de Perú ---

# Filtrar los datos para Perú
peru_data_wide = df_data[df_data['Country Code'] == 'PER'].copy()

if peru_data_wide.empty:
    print("\nError: No se encontraron datos para Perú (Código PER). Verifica los archivos.")
    exit()

# Eliminar columnas innecesarias para la transformación
cols_to_drop = ['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code']
# También eliminar columnas que no sean años (si las hubiera al final, como 'Unnamed: 68')
cols_years = [col for col in peru_data_wide.columns if col.isdigit()] # Asume que los años son columnas nombradas como números

peru_data_wide = peru_data_wide[cols_years]

# Transformar de formato ancho (wide) a largo (long)
# 'melt' toma las columnas de años y las convierte en filas
peru_data_long = peru_data_wide.melt(var_name='Year', value_name='Net Migration')

# Convertir las columnas a tipos de datos adecuados
peru_data_long['Year'] = pd.to_numeric(peru_data_long['Year'])
# Intentar convertir 'Net Migration' a numérico, los errores se volverán NaN (Not a Number)
peru_data_long['Net Migration'] = pd.to_numeric(peru_data_long['Net Migration'], errors='coerce')

# Ordenar por año para graficar correctamente
peru_data_long = peru_data_long.sort_values('Year')

print("\n--- Primeras filas de datos procesados para Perú ---")
print(peru_data_long.head())
print("\n--- Últimas filas de datos procesados para Perú ---")
print(peru_data_long.tail())


# --- PASO 4: Limpieza - Manejo de Valores Faltantes ---
missing_values = peru_data_long['Net Migration'].isnull().sum()
print(f"\nNúmero de valores faltantes en 'Net Migration' para Perú: {missing_values}")

# Opcional: Si hay valores faltantes, podrías decidir imputarlos (rellenarlos).
# Para series de tiempo, la interpolación lineal suele ser una opción razonable.
# peru_data_long['Net Migration'] = peru_data_long['Net Migration'].interpolate(method='linear')
# print("Valores faltantes imputados usando interpolación lineal (si los había).")

# Por ahora, solo los identificamos. Matplotlib a menudo puede graficar con NaNs presentes (dejando huecos).

# --- PASO 5: Análisis Descriptivo Básico ---
print("\n--- Estadísticas Descriptivas de Migración Neta para Perú ---")
print(peru_data_long['Net Migration'].describe())

# Encontrar año con máxima emigración (valor más negativo) y máxima inmigración (valor más positivo)
max_emigration_year = peru_data_long.loc[peru_data_long['Net Migration'].idxmin()]
max_immigration_year = peru_data_long.loc[peru_data_long['Net Migration'].idxmax()]

print(f"\nMáxima emigración neta registrada: {max_emigration_year['Net Migration']:.0f} en el año {int(max_emigration_year['Year'])}")
print(f"Máxima inmigración neta registrada: {max_immigration_year['Net Migration']:.0f} en el año {int(max_immigration_year['Year'])}")


# --- PASO 6: Visualización ---

plt.style.use('seaborn-v0_8-whitegrid') # Estilo de gráfico agradable

# Gráfico 1: Serie de Tiempo de Migración Neta
plt.figure(figsize=(14, 7)) # Tamaño del gráfico
sns.lineplot(data=peru_data_long, x='Year', y='Net Migration', marker='o', color='royalblue')
plt.axhline(0, color='red', linestyle='--', linewidth=0.8) # Línea en cero para ver cambio emigración/inmigración
plt.title('Migración Neta Anual en Perú (1960-2023)', fontsize=16)
plt.xlabel('Año', fontsize=12)
plt.ylabel('Migración Neta (Personas)', fontsize=12)
plt.grid(True)
plt.show()

# Gráfico 2: Histograma de los valores de Migración Neta
plt.figure(figsize=(10, 6))
sns.histplot(data=peru_data_long, x='Net Migration', kde=True, bins=15, color='skyblue') # kde=True añade una línea de densidad
plt.title('Distribución de la Migración Neta Anual en Perú', fontsize=16)
plt.xlabel('Migración Neta (Personas)', fontsize=12)
plt.ylabel('Frecuencia (Número de Años)', fontsize=12)
plt.grid(True, axis='y')
plt.show()

# Gráfico 3: Box Plot (Diagrama de Caja) de la Migración Neta
plt.figure(figsize=(8, 6))
sns.boxplot(data=peru_data_long, y='Net Migration', color='lightgreen')
plt.title('Diagrama de Caja de la Migración Neta Anual en Perú', fontsize=16)
plt.ylabel('Migración Neta (Personas)', fontsize=12)
plt.grid(True, axis='y')
plt.show()

# --- PASO 7: Interpretación Inicial (Ejemplos) ---
print("\n--- Interpretación Inicial (Basada en Gráficos y Estadísticas) ---")
print("1. Tendencia General: Observa el gráfico de línea. ¿Perú ha sido históricamente un país de emigración (valores < 0) o inmigración (valores > 0)? ¿Ha habido cambios significativos en esta tendencia?")
print("2. Picos y Valles: ¿En qué años se observan los picos de emigración (valles más profundos) o inmigración (picos más altos)? Estos podrían correlacionarse con eventos históricos, económicos o políticos.")
print("3. Distribución: El histograma y el boxplot muestran cómo se agrupan los valores anuales. ¿Son la mayoría de los años de emigración moderada/alta? ¿Hay años con valores muy extremos (outliers)?")
print(f"   - La media ({peru_data_long['Net Migration'].mean():.0f}) y la mediana ({peru_data_long['Net Migration'].median():.0f}) dan una idea del valor 'típico' anual.")
print(f"   - El rango va desde {peru_data_long['Net Migration'].min():.0f} hasta {peru_data_long['Net Migration'].max():.0f}.")

In [ ]:
# Seleccionar países de interés (Perú y vecinos)
country_codes_heatmap = ['PER', 'ECU', 'COL', 'BRA', 'BOL', 'CHL']
heatmap_data = df_data[df_data['Country Code'].isin(country_codes_heatmap)].copy()

# Mantener solo columnas relevantes: Country Name y los años
cols_years_heatmap = ['Country Name'] + cols_years # Reutilizamos la lista de años ya identificada
heatmap_data = heatmap_data[cols_years_heatmap]

# Establecer 'Country Name' como índice para el heatmap
heatmap_data = heatmap_data.set_index('Country Name')

# Convertir las columnas de años a numérico (ya deberían serlo, pero por si acaso)
heatmap_data[cols_years] = heatmap_data[cols_years].apply(pd.to_numeric, errors='coerce')

# --- PASO ADICIONAL: Visualización - Mapa de Calor Comparativo ---

plt.figure(figsize=(18, 6)) # Ajustar tamaño según necesidad

# Crear el heatmap
sns.heatmap(
    heatmap_data,           # Los datos (Países x Años)
    cmap="coolwarm",        # Paleta de colores: cálido para positivo (inmigración), frío para negativo (emigración)
    center=0,               # Centrar la escala de colores en 0
    annot=False,            # No mostrar los números en cada celda (sería ilegible)
    linewidths=.5,          # Líneas finas para separar celdas
    linecolor='lightgray'   # Color de las líneas
)

plt.title('Comparación de Migración Neta: Perú y Países Vecinos (1960-2023)', fontsize=16)
plt.xlabel('Año', fontsize=12)
plt.ylabel('País', fontsize=12)
plt.xticks(rotation=90) # Rotar etiquetas de año si son muchas
plt.yticks(rotation=0) # Mantener nombres de países horizontales
plt.tight_layout() # Ajustar para evitar solapamientos
plt.show()

# --- Interpretación del Heatmap (Ejemplos a buscar) ---
print("\n--- Interpretación del Mapa de Calor (Ejemplos a buscar) ---")
print("1. Patrones Similares/Diferentes: ¿Muestran los países vecinos patrones similares a Perú (ej. emigración histórica, inmigración reciente)? ¿O son muy diferentes?")
print("2. Sincronización: ¿Ocurren los picos de emigración/inmigración en los mismos años en varios países (sugiriendo causas regionales/globales) o son específicos de cada país?")
print("3. Magnitud Comparativa: Aunque los números exactos no están, los colores dan una idea de la magnitud relativa. ¿Los flujos de Perú son grandes o pequeños en comparación con Brasil o Colombia, por ejemplo?")
print("4. Caso Venezuela (Contexto): El fuerte color cálido (inmigración) en Perú, Colombia y Ecuador en los últimos años probablemente refleja el impacto regional de la migración venezolana.")